# TEST

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from tools.network import Network
from tools.decode_raw import decode_img_seg, decode_img_seg_test

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train')

feature_dict = {
    'img': 'bytes', 
    'label': 'bytes',
    'height': 'int', 
    'width': 'int',
    'channels': 'int',
    'n_class': 'int'
}

network = Network()

In [3]:
seg_train_gen_params = {
    'csv_path': csv_path,
    'train_path': train_path,
    'height': 256,
    'width': 1600, 
    'col': False,
    'sep':'[_,]',
    'n_class': 5
}

gen = network.seg_train_gen(**seg_train_gen_params)

write_tfr_params = {
    'data_generator': gen,
    'train_path': train_path,
    'tfrpath': tfr_path,
    'feature_dict': feature_dict,
    'shards': 10,
    'compression': 'GZIP', 
    'c_level': 1
}

network.write_tfr(**write_tfr_params)

100%|██████████| 1255/1255 [01:34<00:00, 13.34it/s]


In [ ]:
rt_params = {
    'feature_dict': feature_dict, 
    'decode_raw': decode_img_seg,
    'tfr_path': os.path.join(tfr_path, '*.tfrecord'),
    'shuffle_buffer': 100,
    'compression': 'GZIP',
    'num_parallel_reads':2
}

readtrain_params = {
    'rt_params': rt_params, 
    'train_path': train_path,
    'epoch': 1,
    'batch_size': 32,
    'reshape': [16, 100],
    'reshape_method': 3}


network.readtrain(**readtrain_params)

In [ ]:
model_params = {
    'num_layers': 3,
    'feature_growth_rate': 16,
    'n_class': 5,
    'channels': 3,
    'padding': 'SAME',
    'dropout_rate':0.25
}

network.model(
    model_name='unet',
    model_params=model_params,
    loss='cross_entropy', 
    metric='neg_dice',
    optimizer='momentun',
    rate=1e-6)

In [ ]:
network.train(ckpt_dir=os.path.join('..', 'tmp', 'ckpt'),
              train_percentage = 0.8, 
              early_stopping = 1,
              verbose = 2,
              retrain=False)

In [ ]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train')
ckpt_dir = os.path.join('..', 'tmp', 'ckpt', 'cv')

feature_dict = {
    'img': 'bytes', 
    'label': 'bytes',
    'height': 'int', 
    'width': 'int',
    'channels': 'int',
    'n_class': 'int'
}

rt_params = {
    'feature_dict': feature_dict, 
    'decode_raw': decode_img_seg,
    'tfr_path': os.path.join(tfr_path, '*.tfrecord'),
    'shuffle_buffer': 100,
    'compression': 'GZIP',
    'num_parallel_reads':2
}

readtrain_params = {
    'rt_params': rt_params, 
    'train_path': train_path,
    'epoch': 2,
    'batch_size': 32,
    'reshape': [32, 200],
    'reshape_method': 3}

arch_params = {
    'num_layers': 3,
    'feature_growth_rate': 16,
    'n_class': 5,
    'channels': 3,
    'padding': 'SAME',
    'dropout_rate':0.25
}

model_params = {
    'model_name': 'unet',
    'model_params': arch_params,
    'loss': 'cross_entropy',
    'metric': 'neg_dice',
    'optimizer': 'momentun',
    'rate': 1e+6}

network = Network()
result = network.cv(
    readtrain_params, 
    model_params,
    ckpt_dir=ckpt_dir,
    nfolds=5,
    train_percentage=0.8,
    early_stopping=1,
    verbose=2)